# Visualization 2

### Geographic Data / Maps

#### Installation
```python
pip3 install --upgrade pip
pip3 install geopandas shapely descartes geopy netaddr
sudo apt install -y python3-rtree
```

- `import geopandas as gpd`
- `.shp` => Shapefile
- `gpd.datasets.get_path(<shp file path>)`:
    - example: `gpd.datasets.get_path("naturalearth_lowres")`
- `gpd.read_file(<path>)`

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import math
import requests
import re
import os

# new import statements
import geopandas as gpd
from shapely.geometry import Point, Polygon, box

In [ ]:
!ls /home/gurmail.singh/.local/lib/python3.8/site-packages/geopandas/datasets/naturalearth_lowres

In [ ]:
!ls /home/gurmail.singh/.local/lib/python3.8/site-packages/geopandas/datasets

In [ ]:
# Find the path for "naturalearth_lowres"
path = 
# Read the shapefile for "naturalearth_lowres" and
# set index using "name" column
gdf = 

In [ ]:
gdf.head()

In [ ]:
type(gdf).__mro__

In [ ]:
# All shapefiles have a column called "geometry"
gdf["geometry"]

In [ ]:
type(gdf["geometry"]).__mro__

In [ ]:
# First country's name and geometry


In [ ]:
# Second country's name geometry
print(gdf.index[1])
gdf["geometry"].iat[1]

In [ ]:
# Geometry for "United States of America"
# gdf.at[<row_index>, <column_name>]

In [ ]:
# Type of Tanzania's geometry
print(gdf.index[1], type(gdf["geometry"].iat[1]))

# Type of United States of America's geometry
print("United States of America", type(gdf.at["United States of America", "geometry"]))

- `gdf.plot(figsize=(<width>, <height>), column=<column name>)`
- `ax.set_axis_off()`

In [ ]:
ax = gdf.plot(figsize=(8,4))

In [ ]:
# Set facecolor="0.7", edgecolor="black"
ax = gdf.plot(figsize=(8,4))
# Turn off the axes
# ax.set_axis_off()

In [ ]:
# Color the map based on population column, column="pop_est" and set cmap="cool" and legend=True
ax = gdf.plot(figsize=(8,4))
# Turn off the axes
ax.set_axis_off()

#### Create a map where countries with >100M people are red, others are gray.

In [ ]:
# Create a map where countries with >100M people are red, others are gray

# Add a new column called color to gdf and set default value to "lightgray"

# Boolean indexing to set color to red for countries with "pop_est" > 1e8

# Create the plot
# ax = gdf.plot(figsize=(8,4), color=gdf["color"])
# ax.set_axis_off()

### All shapefile geometries are shapely shapes. 

In [ ]:
type(gdf["geometry"].iat[2])

### Shapely shapes

- `from shapely.geometry import Point, Polygon, box`
- `Polygon([(<x1>, <y1>), (<x2>, <y2>), (<x3>, <y3>), ...])`
- `box(minx, miny, maxx, maxy)`
- `Point(<x>, <y>)`
- `<shapely object>.buffer(<size>)`
    - example: `Point(5, 5).buffer(3)` creates a circle

In [ ]:
triangle =    # triangle
triangle

In [ ]:
type(triangle)

In [ ]:
box1 =     # not a type; just a function that creates box
box1

In [ ]:
type(box1)

In [ ]:
point = 
point

In [ ]:
type(point)

In [ ]:
# use buffer to create a circle from a point
circle = 
circle

In [ ]:
type(circle)

In [ ]:
triangle_buffer = triangle.buffer(3)
triangle_buffer

In [ ]:
type(triangle_buffer)

#### Shapely methods

- Shapely methods:
    - `union`:  any point that is in either shape (OR)
    - `intersection`: any point that is in both shapes (AND)
    - `difference`: subtraction
    - `intersects`: do they overlap? returns True / False

In [ ]:
# union triangle and box1
# it will give any point that is in either shape (OR)


In [ ]:
# intersection triangle and box1
# any point that is in both shapes (AND)

In [ ]:
# difference of triangle and box1

In [ ]:
# difference of box1 and triangle
box1.difference(triangle)   # subtraction

In [ ]:
# check whether triangle intersects box1
# the is, check do they overlap?

Is the point "near" (<6 units) the triangle?

In [ ]:
triangle.union(point.buffer(6))

In [ ]:
triangle.intersects(point.buffer(6))

#### Extracting "Europe" data from "naturalearth_lowres"

In [ ]:
# Europe bounding box
eur_window = box(-10.67, 34.5, 31.55, 71.05)

Can we use `intersects` method?

In [ ]:
gdf.intersects(eur_window)

In [ ]:
# Incorrect v1
gdf[gdf.intersects(eur_window)].plot()

In [ ]:
# Incorrect v2
gdf[~gdf.intersects(eur_window)].plot()

Can we use `intersection` method?

In [ ]:
gdf.intersection(eur_window)

In [ ]:
gdf.intersection(eur_window).plot()

How can we get rid of empty polygons (and remove the warning)?

In [ ]:
eur = gdf.intersection(eur_window)
eur.is_empty

Remove all the empty polygons using `is_empty`.

In [ ]:
eur = eur[~eur.is_empty]
eur

In [ ]:
eur.plot()

#### Centroids of European countries

In [ ]:
# plot the centroids
ax = eur.plot(facecolor="lightgray", edgecolor="k")
eur.centroid.plot(ax=ax)

### Lat / Lon CRS

- Lon is x-coord
- Lat is y-coord
    - tells you where the point on Earth is
- **IMPORTANT**: degrees are not a unit of distance. 1 degree of longitute near the equator is a lot farther than moving 1 degree of longitute near the north pole

Using `.crs` to access CRS of a gdf.




In [ ]:
eur.crs

#### Single CRS doesn't work for the whole earth

- Setting a different CRS for Europe that is based on meters.
- https://spatialreference.org/ref/?search=europe

In [ ]:
# Setting CRS to "EPSG:3035"
eur2 = eur.to_crs("EPSG:3035")
eur2.crs

In [ ]:
ax = eur2.plot(facecolor="lightgray", edgecolor="k")
eur2.centroid.plot()

In [ ]:
ax = eur2.plot(facecolor="lightgray", edgecolor="k")
eur2.centroid.plot(ax=ax)

#### How much error does lat/long computation introduce?

In [ ]:
ax = eur2.plot(facecolor="lightgray", edgecolor="k")
eur2.centroid.plot(ax=ax, color="k") # black => correct
eur.centroid.to_crs("EPSG:3035").plot(ax=ax, color="r")  # red => miscalculated

In [ ]:
type(eur2.iloc[0])

In [ ]:
type(eur2).__mro__

#### Area of European countries

In [ ]:
eur2.area # area in sq meters

What is the area in **sq miles**?

In [ ]:
# Conversion: / 1000 / 1000 / 2.59
(eur2.area / 1000 / 1000 / 2.59).sort_values(ascending=False)
# careful!  some countries (e.g., Russia) were cropped when we did intersection

In [ ]:
# area on screen, not real area
eur.area

### CRS

- `<GeoDataFrame object>.crs`: gives you information about current CRS.
- `<GeoDataFrame object>.to_crs(<TARGET CRS>)`: changes CRS to `<TARGET CRS>`.

### Madison area emergency services

- Data source: https://data-cityofmadison.opendata.arcgis.com/
    - Search for:
        - "City limit"
        - "Lakes and rivers"
        - "Fire stations"
        - "Police stations"

- CRS for Madison area: https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system#/media/File:Universal_Transverse_Mercator_zones.svg

In [ ]:
city = gpd.read_file("City_Limit.zip").to_crs("epsg:32616")

In [ ]:
city.crs

In [ ]:
water = gpd.read_file("Lakes_and_Rivers.zip").to_crs(city.crs)
fire = gpd.read_file("Fire_Stations.zip").to_crs(city.crs)
police = gpd.read_file("Police_Stations.zip").to_crs(city.crs)

#### Run this on your virtual machine

`sudo sh -c "echo 'Options = UnsafeLegacyRenegotiation' >> /etc/ssl/openssl.cnf"`

then restart notebook!

#### GeoJSON

How to find the below URL?

- Go to info page of a dataset, for example: https://data-cityofmadison.opendata.arcgis.com/datasets/police-stations/explore?location=43.081769%2C-89.391550%2C12.81
- Then click on "I want to use this" > "View API Resources" > "GeoJSON"

In [ ]:
url = "https://maps.cityofmadison.com/arcgis/rest/services/Public/OPEN_DATA/MapServer/2/query?outFields=*&where=1%3D1&f=geojson"
police2 = gpd.read_file(url).to_crs(city.crs)

In [ ]:
ax = city.plot(color="lightgray")
water.plot(color="lightblue", ax=ax)
fire.plot(color="red", ax=ax, marker="+", label="Fire")
police2.plot(color="blue", ax=ax, label="Police")
ax.legend(loc="upper left", frameon=False)
ax.set_axis_off()

In [ ]:
fire.to_file("fire.geojson")

### Geocoding: street address => lat / lon


- `gpd.tools.geocode(<street address>, provider=<geocoding service name>, user_agent=<user agent name>)`: converts street address into lat/long


#### Daily incident reports: https://www.cityofmadison.com/fire/daily-reports

In [ ]:
url = "https://www.cityofmadison.com/fire/daily-reports"
r = requests.get(url)
r

In [ ]:
r.raise_for_status() # give me an exception if not 200 (e.g., 404)

In [ ]:
# doesn't work
# pd.read_html(url)

In [ ]:
# print(r.text)

Find all **span** tags with **streetAddress** using regex.

In [ ]:
# <p>1700 block Thierer Road<br>
# addrs = re.findall(r'<p>1700 block Thierer Road<br>', r.text)

In [ ]:
addrs = re.findall(r'', r.text)
addrs = pd.Series(addrs)
addrs

#### Without city name and state name, geocoding would return match with the most famous location with such a street name.

In [ ]:
geo_info = gpd.tools.geocode("1300 East Washington Ave")
geo_info

In [ ]:
geo_info["address"].loc[0]

#### To get the correct address we want, we should concatenate "Madison, Wisconsin" to the end of the address.

In [ ]:
geo_info = gpd.tools.geocode("1300 East Washington Ave, Madison, Wisconsin")
geo_info

#### Addresses with "block" often won't work or won't give you the correct lat/long. We need to remove the word "block" before geocoding.

In [ ]:
gpd.tools.geocode("800 block W. Johnson Street, Madison, Wisconsin")

In [ ]:
gpd.tools.geocode("800 W. Johnson Street, Madison, Wisconsin")

In [ ]:
fixed_addrs = addrs.str.replace(" block ", " ") + ", Madison, WI"
fixed_addrs

#### Using a different provider than the default one

- `gpd.tools.geocode(<street address>, provider=<geocoding service name>, user_agent=<user agent name>)`: converts street address into lat/long
    - We will be using "OpenStreetMap", for which the argument is "nominatim"
    - We also need to specify argument to `user_agent` parameter, indicating where the request is coming from; for example: "cs320_bot"

In [ ]:
incidents = gpd.tools.geocode(fixed_addrs, provider="nominatim", user_agent="cs320bot")
incidents

It is often a good idea to drop na values. Although in this version of the example, there are no failed geocodings.

In [ ]:
incidents = incidents.dropna()
incidents

#### Self-practice

If you want practice with regex, try to write regular expression and use the match result to make sure that "Madison" and "Wisconsin" is part of each address.

In [ ]:
# self-practice
for addr in incidents["address"]:
    print(addr)

In [ ]:
ax = city.plot(color="lightgray")
water.plot(color="lightblue", ax=ax)
fire.plot(color="red", ax=ax, marker="+", label="Fire")
police2.plot(color="blue", ax=ax, label="Police")
incidents.to_crs(city.crs).plot(ax=ax, color="k", label="Incidents")
ax.legend(loc="upper left", frameon=False)
ax.set_axis_off()